### Distribution function for the DM mini-halo

The idea is the construct an equilibrium distribution function for a DM mini-halo (around a PBH) with a density profile $\rho \sim r^{-3/2}$ within the truncation radius $r_\mathrm{tr}$. 

In [ ]:
import numpy as np
import sympy as sp

from tqdm import tqdm
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
from scipy.integrate import quad
from scipy.special import betainc, beta, btdtr,hyp2f1

from sympy.assumptions import assuming, Q

from __future__ import division

from matplotlib import pylab as pl
import matplotlib as mpl
mpl.rc('font', **{'size':18})

mpl.rcParams['xtick.major.size'] = 7
mpl.rcParams['xtick.major.width'] = 1
mpl.rcParams['xtick.minor.size'] = 3
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.size'] = 3
mpl.rcParams['ytick.minor.width'] = 1

def beta_incomplete(a,b,x):
    return (x**a/a)*hyp2f1(a,1-b,a+1,x)

**Parameters:**

In [ ]:
MPBH = 30

avals, rvals, zvals = np.loadtxt("distributions/Decoupling_M=" + str(int(MPBH)) + "Msun.txt", unpack=True)
rtr_interp = interp1d(avals, rvals)

#Specify the value of a!
a = 0.002
rtr = rtr_interp(a)
print a/rtr
print "Truncation radius [pc]:", rtr
#print rtr_interp(0.0896960135445)

r_eq = 0.0063*(MPBH**(1.0/3.0)) #Truncation radius in pc at equality
G = 4.302e-3 #Units of (pc/solar mass) (km/s)^2
A = 3.0*MPBH/(8*np.pi*(rtr*r_eq)**1.5)

print (rtr/r_eq)**(-3/2.0)

print MPBH*(rtr/r_eq)**(3/2.0)

**DM Density profile:**

In [ ]:
x_a = 1.0
rho0 = 1.0

def rho_inner(x):
    return 1.0/((x**(3/2))*(1 + x)**(6-3/2))    

alpha = 1.0*rho_inner(x_a)
#beta = (1.0/alpha)*1.0*3*(1+4*x_a)/(2*x_a**(5/2)*(1+x_a)**(11/2))
beta = 3*(1+4*x_a)/(2.0*x_a*(1+x_a))

def rho_outer(x):
    return alpha*sp.exp(-1.0*beta*(x-x_a))

def rho(x):
    if (x < x_a):
        return rho_inner(x)
    if (x >= x_a):
        return rho_outer(x)

def P(x):
    return 4*np.pi*1.0*x**(1/2)/((1 + x)**(4-3/2))

def rho_true(x):
    if (x > 1.0):
        return 0.0
    return A*x**(-1.5)

print quad(lambda r: 4*np.pi*rho_true(r/rtr)*r**2, 0, rtr)[0]

rho_vec = np.vectorize(rho)


**Plot the density...**

In [ ]:
#Check the density
pl.figure(figsize=(7,5))
xlist = np.logspace(np.log10(1e-5), np.log10(10),1000)
pl.loglog(xlist, np.vectorize(rho_true)(xlist), label='UCMH (Eq. 13)', lw=1.5, color='C3')
pl.loglog([1.0, 1.0], [1e-3, rho_true(1.0)],lw=1.5, color='C3')
pl.loglog(xlist, rho0*np.vectorize(rho)(xlist),linestyle='--',label='gNFW', lw=1.5, color='C9')
#pl.loglog(xlist, rho_inner(xlist))
pl.xlabel(r"$x = r/R_\mathrm{tr}$")
pl.ylabel(r"$\rho_\mathrm{DM}$ [$M_\odot$ pc$^{-3}$]")
pl.ylim(1e5, 1e15)
pl.xlim(1e-4, 10)
pl.axvline(1.0, linestyle=':', color='k')
#pl.axvline(x_b, linestyle=':', color='k')
pl.legend(loc="best",fontsize=16.0)
#pl.xticks([1e-3, 1e-2, 1e-1, 1, 10], [r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$',r'$1$',r'$10$'])

pl.tight_layout()
pl.savefig("../plots/Halo_density.pdf",bbox_inches="tight")
pl.show()

#Check the density
pl.figure()
xlist = np.logspace(np.log10(1e-3), np.log10(10),1000)
pl.loglog(xlist, np.vectorize(rho)(xlist)/np.vectorize(rho_true)(xlist),label='gNFW')
#pl.loglog(xlist, rho_inner(xlist))
pl.xlabel(r"$x = r/r_\mathrm{tr}$")
pl.ylabel(r"$\rho_\mathrm{DM}$ [$M_\odot$ pc$^{-3}$]")
#pl.ylim(0, 1e6)
#pl.xlim(0.5, 5)
pl.axvline(1.0, linestyle=':', color='k')
#pl.axvline(x_b, linestyle=':', color='k')
#pl.legend(loc="best",fontsize=10.0)
#pl.tight_layout()
#pl.savefig("../../Halo_density.pdf",bbox_inches="tight" )
pl.show()

print rho0*rho(1e-5)/rho_true(1e-5)

In [ ]:
#Check the density
pl.figure()
xlist = np.logspace(np.log10(1e-3), np.log10(3),1000)
pl.semilogy(xlist, 4*np.pi*np.vectorize(rho)(xlist)*xlist**2)
pl.semilogy(xlist, 4*np.pi*np.vectorize(rho_true)(xlist)*xlist**2)
pl.xlabel(r"$x = r/r_\mathrm{tr}$")
pl.ylabel(r"$\rho_\mathrm{DM}$ [$M_\odot$ pc$^{-3}$]")
pl.ylim(1e5, 1e8)
pl.show()

**Calculate masses and potentials:**

In [ ]:
x1,x2,x3,y = sp.symbols('x1 x2 x3 y')
rho0 = 1.0

def Menc_inner(x):
    return rtr**3*4*np.pi*1.0*2*(x**(3/2))*(35 + 28*x + 8*x**2)/(105.0*(1+x)**(7/2))

Mouter_fun = Menc_inner(x_a) + rtr**3*4*np.pi*sp.integrate(rho_outer(x1)*x1**2, (x1, x_a, x2))
#print Menc_inner(0.5)
#print Mouter_fun.subs(x1,1.5)
rho0 = ((rtr/r_eq)**1.5)*MPBH/Mouter_fun.subs(x2, 100)
print rho0

def Menc(x):
    if (x < x_a):
        return MPBH + rho0*Menc_inner(x)
    elif (x >= x_a):
        return MPBH + rho0*Mouter_fun.subs(x2, x)    

def psi_outer(y):
    return (MPBH + rho0*Menc_inner(x_a))*G/(rtr*y) - (2**(-9/2))*4*np.pi*rtr**3*(G/rtr)*rho0*(np.exp((x_a - y)*beta)*alpha*(2 -2*np.exp(y*beta) + y*beta)/(y*beta**3))
    
def psi_inner1(y):
    num = 48-70*np.sqrt(y*(1+y)) - 112*np.sqrt(y**3*(1+y))+ \
            -48*np.sqrt(y**5*(1+y)) + 48*y*(3+y*(3+y))
    return (rtr**3*4*np.pi*rho0)*(G/rtr)*2*num/(105.0*(1+y)**3) + MPBH*G/(rtr*y)



def psi_inner(y):
    return psi_inner1(y) - psi_inner1(x_a) + psi_outer(x_a)

def psi(y):
    if (y < x_a):
        return psi_inner(y)
    elif (y >= x_a):
        return psi_outer(y)
    
psi_tr = float(psi(1.0))
psi_min = float(psi(100))
psi_max = float(psi(1e-6))



**Plot the potential and mass enclosed:**

In [ ]:
#Check the potential
print "Potential at x = 100:", psi_min
#print "Potential at x = 1.1:", psi_b
print "Potential at x = 1:", psi_tr
print "Potential at x = 1e-5:", psi_max
print " "
print "Mass enclosed at x = 1:", Menc(1.0)-30
print "Mass enclosed at x = 1.5:", Menc(1.5)-30
print "Mass enclosed at x = 2:", Menc(2)-30
print "Mass enclosed at x = 5:", Menc(5)-30
print "Mass enclosed between at x = 5 and x = 8:", Menc(8)-Menc(5)
print " "
print "Fraction of particles inside x = 1e-2:", (Menc(1e-2)-30)/(Menc(100)-30)

pl.figure()
xlist = np.linspace(1e-3, 5, 100)
pl.loglog(xlist, np.vectorize(psi)(xlist))
pl.xlabel(r"$x = r/r_\mathrm{tr}$")
pl.ylabel(r"$\psi$ [$(\mathrm{km}/\mathrm{s})^2$]")
#pl.axvline(x_a, linestyle=':', color='k')
pl.axvline(1.0, linestyle=':', color='k')
pl.xlim(0.1, 2)
pl.show()

pl.figure()
xlist = np.linspace(0.0, 5, 100)
pl.plot(xlist, np.vectorize(Menc)(xlist))
pl.xlabel(r"$x = r/r_\mathrm{tr}$")
pl.ylabel(r"$M_\mathrm{enc} [M_\odot]$")
pl.axvline(1.0, linestyle=':', color='k')
#pl.axvline(x_b, linestyle=':', color='k')
pl.show()

### Calculating moment of inertia:

In [ ]:
I = 4*np.pi*rho0*rtr**5*quad(lambda x: rho(x)*x**4, 0, 5)[0]
print I

#print rtr, r0

#r0 = 0.5*((1+0.9)*a)
#L0 = MPBH*r0**2
#L1 = ((MPBH+3.1)*r0**2 + I)

#ratio = ((3.1)*r0**2 + I)

#print L0, L1, (ratio)

#e = 0.9
#j = np.sqrt(1-e**2)
#Lsq = 0.5*G*(Menc(10)**3)*0.063*j**2
#print Lsq**0.5

#### Calculating the binding energy

In [ ]:
def U_integ(r):
    return Menc(r/rtr)*4.*np.pi*r**2*rho0*rho(r/rtr)/r

def U_integ_new(r):
    if (r < rtr):
        M1 = MPBH*(1 + (r/r_eq)**1.5)
    else:
        M1 = MPBH*(1 + (rtr/r_eq)**1.5)
    #print r/rtr, rho_true(r/rtr)
    return M1*4.*np.pi*r**2*rho_true(r/rtr)/r


print "Old:", U_integ(1e-1*rtr)
print "New:", U_integ_new(1e-1*rtr)


#print 4*np.pi*rho0*quad(lambda r: r**2*rho(r/rtr), 0, 5*rtr)[0]

U_bind = -G*quad(U_integ, 0, 5*rtr)[0]

#U_bind = -G*quad(U_integ, 1e-5, 10*rtr)[0]

print "Binding energy of the halo:", U_bind

#Binding energy of the PBH+halo pair

a_final = 0.0078196

U_orb_before = -G*(Menc(10)**2)/(2.0*a)
U_orb_after = -G*(MPBH**2)/(2.0*a_final)

print "Orbital energy of the PBH+halo pair (a = " + str(a) + "):", U_orb_before

print "Orbital energy of the bare PBH pair (a = " + str(a_final) + "):", U_orb_after

print " "
print "Total energy before:", U_orb_before + 2.0*U_bind
print "Total energy after:", U_orb_after

print "  "
print "Predicted final semi-major axis:", -G*MPBH**2*0.5/(U_orb_before + 2.0*U_bind)



In [ ]:
rmin_list = np.logspace(-9, 1,50)
print 0.5*rmin_list
print 2*rtr
Ulist = np.asarray([-G*quad(U_integ, 0.5*rmin, 5*rtr)[0] for rmin in rmin_list])
Uinterp = interp1d(rmin_list, Ulist)

In [ ]:
Menc_list = np.vectorize(Menc)(0.5*rmin_list/rtr)
#print Menc_list

a_f_list = -G*Menc_list**2*0.5/(U_orb_before + 2.0*Ulist)

pl.figure()
pl.semilogx(rmin_list, Ulist)
pl.xlabel(r"$r_\mathrm{min}$ [pc]")
pl.ylabel(r"Binding energy down to $r_\mathrm{min}$")
pl.show()

pl.figure()
pl.semilogx(rmin_list, a_f_list)
pl.xlabel(r"$r_\mathrm{min}$ [pc]")
pl.ylabel(r"Expected final $a$ [pc]")
pl.show()

jlist = np.logspace(-3, 0,200)
rmin_vals = a*(1-np.sqrt(1-jlist**2))
Menc_vals = np.vectorize(Menc)(0.5*rmin_vals/rtr)
a_f_vals = -G*Menc_vals**2*0.5/(U_orb_before + 2.0*Uinterp(rmin_vals))

#print -G*MPBH**2*0.5/(U_orb_before + 2.0*Uinterp((1-0.95)*(0.063)))

pl.savetxt("testj_M" + str(int(MPBH))+ "_a" + str(a) + ".txt", zip(jlist, a_f_vals))

pl.figure()
pl.loglog(jlist, a_f_vals)
pl.xlabel(r"$j_i$")
pl.ylabel(r"Expected final $a$ [pc]")
pl.show()

In [ ]:
def calc_af(ai):
    global rtr
    rtr = rtr_interp(ai)
    U_orb_before = -G*(Menc(10)**2)/(2.0*ai)
    Ubind = -G*quad(U_integ, 1e-8, 5.0*rtr, epsrel=1e-3)[0]
    return -G*MPBH**2*0.5/(U_orb_before + 2.0*Ubind)

def calc_af_true(ai):
    global rtr
    
    rtr = rtr_interp(ai)
    #print rtr

    M1 = MPBH*(1 + (rtr/r_eq)**1.5)
    U_orb_before = -G*(M1**2)/(2.0*ai)
    
    Ubind = -G*quad(U_integ_new, 1e-8, 1.0*rtr, epsrel=1e-5)[0]
    return -G*MPBH**2*0.5/(U_orb_before + 2.0*Ubind)

print -G*quad(U_integ, 0, 10.0*rtr, epsrel=1e-3)[0]
print -G*quad(U_integ_new, 0, 1.0*rtr, epsrel=1e-3)[0]

rvals = np.logspace(-6, 0)
pl.figure()
pl.loglog(rvals, np.vectorize(U_integ)(rvals), label="gNFW")
pl.loglog(rvals, np.vectorize(U_integ_new)(rvals), label="UCMH")
pl.legend()
pl.ylim(1e-20, 1e20)
pl.show()


In [ ]:
print calc_af(0.01)
print calc_af_true(0.01)

In [ ]:
ai_list = np.logspace(-5, -1)
af_list = 0.0*ai_list
af_true_list = 0.0*ai_list

for i in tqdm(range(len(ai_list))):
    af_list[i] = calc_af(ai_list[i]) 
    af_true_list[i] = calc_af_true(ai_list[i])

    


In [ ]:
#ai_of_af = UnivariateSpline(af_list,ai_list,k=1,s=0)


pl.figure()
pl.loglog(ai_list, af_list, label='NFW')
pl.loglog(ai_list, af_true_list, label='UCMH')
#pl.loglog(ai_of_af(af_list), af_list, '+')
pl.loglog([1e-5, 1e-1], [1e-5, 1e-1], color='k', linestyle='--')
pl.xlabel(r'$a_i$')
pl.ylabel(r'$a_f$')
pl.title(r'$M_\mathrm{PBH} = ' + str(MPBH)+'\,M_\odot$', fontsize=16)
pl.legend(loc='best')
pl.show()

pl.figure()
pl.loglog(af_list, ai_list)
#pl.loglog(ai_of_af(af_list), af_list, '+')
pl.loglog([1e-5, 1e-1], [1e-5, 1e-1], color='k', linestyle='--')
pl.xlabel(r'$a_f$')
pl.ylabel(r'$a_i$')
pl.title(r'$M_\mathrm{PBH} = ' + str(MPBH)+'\,M_\odot$', fontsize=16)

pl.show()


In [ ]:
af_0 = 1.0*calc_af_true(0.01)
print af_0

def M_halo(a, M_PBH=30.0):
    return M_PBH*(rtr_interp(a)/r_eq)**1.5

def calcL(M, a ,j):
    return np.sqrt(0.5*G*M**3*a*j**2)

def calcFinalj(a_ini, j_ini, kappa, M_PBH):
    #a_fin = a_final_interp(j_ini)

    a_fin = 1.0*calc_af_true(a_ini)
    Mh = M_halo(a_ini, M_PBH)
    L_ini = calcL(M_PBH + Mh, a_ini,j_ini)
    LDM_final = kappa + L_ini*Mh/(M_PBH + Mh)
    #LDM_final = kappa
    LPBH_final = np.clip(L_ini - LDM_final, 0, L_ini)
    #print LPBH_final
    return LPBH_final*(2.0/(G*M_PBH**3*a_fin))**0.5

In [ ]:
a1 = 0.063
j1 = 1e-5

print (a1**4*(j1)**7)
print (calc_af_true(a1)**4*(calcFinalj(a1,j1, 0.0, 1000.0))**7)

**Calculating density as a function of potential:**

In [ ]:
#List of x values
#xlist = np.logspace(-4, np.log10(x_a-0.06),1000)
#xlist = np.append(xlist,np.linspace(x_a-0.05, x_b+0.05),1000)
#xlist = np.append(xlist,np.linspace(x_b+0.06,20,1000))
#xlist = np.sort(xlist)
xlist = np.append(np.logspace(-5, 0, 2000),np.linspace(1.10001, 10.0, 1000))
xlist = np.append(xlist, np.linspace(10.1, 100.0, 1000))
#xlist = np.append(np.logspace(-5, 1, 10000), 1.0)

xlist = np.append(xlist, np.linspace(0.9, 1.1, 1000))
xlist = np.sort(xlist)


In [ ]:
rholist = np.vectorize(rho)(xlist)
psilist = np.asarray(np.vectorize(psi)(xlist),dtype='float64')
rholist = np.append(rholist, 0)
psilist = np.append(psilist, 0)
rho_of_psi = UnivariateSpline(psilist[::-1],rholist[::-1],k=5,s=0)

In [ ]:

psi_interp = UnivariateSpline(xlist,psilist[:-1],k=5,s=0)
dpsi = psi_interp.derivative(n=1)
pl.figure()
pl.loglog(xlist,-dpsi(xlist))
#pl.semilogy(psilist,alpha1*np.exp(1.1/beta1)*np.exp(-c/(psilist*beta1))*c/(beta1*psilist**2))
#pl.axvline(psi_min, color='k', linestyle=":")
pl.axvline(1.0, color='k', linestyle=":")
pl.xlim(0.1, 10.0)
pl.ylim(1e-1, 1e2)

pl.show()

In [ ]:
pl.figure()
pl.loglog(psilist,rholist)
pl.loglog(psilist,rho_of_psi(psilist))

pl.axvline(psi_min, color='k', linestyle=":")
pl.axvline(psi_tr, color='k', linestyle=":")
#pl.xlim(psi_tr - 5,psi_tr + 10)
#pl.ylim(1e-20, 1e10)
pl.show()

In [ ]:
drho = rho_of_psi.derivative(n=1)
d2rho = rho_of_psi.derivative(n=2)

In [ ]:
c = 14.2844298406921
def d2rho_fixed_scalar(p):
    delta = 0.1
    if (p < -1):
        return 0.0
    elif (psi_tr-delta < p <= psi_tr):
        return d2rho(psi_tr - delta)
    elif (psi_tr < p < psi_tr + delta):
        return d2rho(psi_tr + delta)
    elif (p >= -1):
        return d2rho(p)

d2rho_fixed = np.vectorize(d2rho_fixed_scalar)

In [ ]:
d2rho(10)

In [ ]:
pl.figure()
pl.loglog(psilist,drho(psilist))
#pl.semilogy(psilist,alpha1*np.exp(1.1/beta1)*np.exp(-c/(psilist*beta1))*c/(beta1*psilist**2))
pl.axvline(psi_min, color='k', linestyle=":")
pl.axvline(psi_tr, color='k', linestyle=":")
pl.xlim(psi_min, 2*psi_tr)
pl.ylim(1e-3, 1e-1)
pl.show()

In [ ]:
pl.figure()
pl.loglog(psilist,d2rho(psilist), alpha=0.25)
pl.loglog(psilist,d2rho_fixed(psilist))
#pl.plot(psilist,d2rho_fixed(psilist)/np.sqrt(15.0 - psilist))
#pl.plot(psilist,np.vectorize(d2rho_fixed)(psilist))
pl.xlim(psi_tr - 1.0, psi_tr+1.0)
pl.ylim(1e-3, 0.01)
pl.axvline(psi_min, color='k', linestyle=":")
pl.axvline(psi_max, color='k', linestyle=":")
pl.axvline(psi_tr, color='k', linestyle=":")
pl.show()



In [ ]:
def calcf1(eps):
    #if (eps <= psi_min):
    #    return 0.0
    if (eps < 1e-10):
        return 0.0
    integ = lambda p: rho0*d2rho_fixed_scalar(p)/np.sqrt(eps - p)
    return quad(integ, 0, eps)[0]/(np.sqrt(8)*np.pi**2)

In [ ]:
#e_list = np.append(0.0,np.logspace(np.log10(psi_min), np.log10(psi_max),1000))
e_list = np.append(np.logspace(np.log10(psi_min),np.log10(psi_tr-0.0001), 300),np.logspace(np.log10(psi_tr+0.0001), np.log10(psi_max),300))
#e_list = np.linspace(0.0, 14, 200)
e_list = np.sort(np.append(e_list, np.linspace(psi_tr-1.0, psi_tr+1.0, 100)))
e_list = np.append(0, e_list)
#e_list = np.linspace(0.0, psi_max, 1000)

In [ ]:
fvals = 0.0*e_list
for i in tqdm(range(len(e_list))):
    fvals[i] = calcf1(e_list[i])

#print fvals

In [ ]:
#e_list, fvals = np.loadtxt("distribution_trunc3.dat", unpack=True)

In [ ]:
finterp = UnivariateSpline(e_list, fvals, k=1,s=0)
def f(eps):
    #if (eps < 1):
    #    return 0.0
    return np.clip(finterp(eps),0,1e30)

f_vec = np.vectorize(f)

In [ ]:
pl.figure()
pl.loglog(e_list,f_vec(e_list))
#pl.semilogx(e_list,np.vectorize(stepfun)(e_list))
#pl.axvline(psi_tr, linestyle=':', color='k')
#pl.ylim(1, 1e4)
#pl.xlim(0, 20.0)
#print psi(5)
pl.axvline(psi_min, color='k', linestyle=":")
pl.axvline(psi_tr, color='k', linestyle=":")
pl.show()

**Re-checking the density:**

In [ ]:
def vmax(x):
    return np.sqrt(2*float(psi(x)))

def dens(x):
    integ = lambda v: v**2*f(float(psi(x)) - 0.5*v**2)
    return 4*np.pi*quad(integ, 0, vmax(x))[0]

In [ ]:
#print float(psi(2.0))
print psi(0.75)
print dens(0.75)/(rho0*rho(0.75))

pl.figure()
pl.plot(e_list,f_vec(e_list))
pl.plot(e_list,fvals,'r+')
#pl.semilogx(e_list,np.vectorize(stepfun)(e_list))
#pl.axvline(psi_tr, linestyle=':', color='k')
#pl.ylim(1e-10, 1000)
#pl.xlim(10, 30)
pl.show()

pl.figure()
pl.loglog(e_list,f_vec(e_list))
pl.loglog(e_list,fvals,'r+')
#pl.semilogx(e_list,np.vectorize(stepfun)(e_list))
pl.axvline(psi_tr, linestyle=':', color='k')
#pl.xlim(1, 16.36)
pl.show()

In [ ]:
xlist2 = np.logspace(np.log10(1e-6), np.log10(10),50)
#xlist2 = np.linspace(0.85, 1.15,10)
dens_list = 0.0*xlist2
for i in tqdm(range(len(xlist2))):
    dens_list[i] = dens(xlist2[i])

In [ ]:
print psi(1.7)
print rho0*rho(0.8)

pl.figure()
pl.loglog(xlist2, rho0*np.vectorize(rho)(xlist2), label="True")
pl.loglog(xlist2, dens_list, label="Reconstructed")
pl.legend(loc='best',fontsize=12.0)
pl.xlabel(r'$x = r/r_\mathrm{tr}$')
pl.ylabel(r"$\rho_\mathrm{DM}$ [$M_\odot$ pc$^{-3}$]")

pl.figure()
pl.semilogx(xlist2, (1/rho0)*dens_list/np.vectorize(rho)(xlist2))
pl.xlabel(r'$x = r/r_\mathrm{tr}$')
pl.ylabel(r"$\rho_\mathrm{recon}/\rho_\mathrm{true}$")
#pl.ylim(0.99,1.01)
#pl.xlim(0, 2)
pl.show()

In [ ]:
htxt = "M_PBH = " + str(int(MPBH)) + " M_solar, a/pc = "+"{0:.3f}".format(a) + ", r_tr/pc = " + str(rtr)
outfilename = "distribution_M=" + str(int(MPBH)) + "_a=" +"{0:.3f}".format(a) + ".dat"

np.savetxt(outfilename,zip(e_list,np.clip(fvals,0,1e30)), header=htxt)

print outfilename

In [ ]:
#Speed distribution f(v) at a given radius r
def f_scalar(x, v):
    if (v >= vmax(x)):
        return 0.0
    else:
        return 4.0*np.pi*(v**2)*f(float(psi(x)) - 0.5*v**2)/(rho0*rho(x))

f_v = np.vectorize(f_scalar)

In [ ]:
xpos = 1e-6
vlist = np.linspace(0, vmax(xpos),100)
#upper = 4*np.pi*vmax(xpos)**2*np.max(fvals[e_list < float(psi(xpos))])/rho(xpos)
#print upper
pl.figure()
pl.plot(vlist, f_v(xpos,vlist))
#pl.axhline(upper, linestyle=':', color='g')
pl.axhline(5.0/vmax(xpos), linestyle=':', color='r')
pl.show()


In [ ]:
print "DONE!"


#pl.figure(figsize=(7,7))

#for astr in ["0.001","0.005", "0.010", "0.020", "0.040", "0.080"]:
#    x,y = np.loadtxt("distribution_a=" + astr + ".dat", unpack=True)
#    pl.loglog(x,y, label = "a = " + astr)

#pl.ylim(1e-5, 1e5)
#pl.legend(loc='lower right')
#pl.show()